# Python

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

In [ ]:
%%shell
mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

# R

In [ ]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

In [ ]:
require("data.table")

if(!require("R.utils")) install.packages("R.utils", repos = "http://cran.us.r-project.org")
require("R.utils")

In [ ]:
# -----------------------------------------------------------------------------
# 1. DEFINICIÓN DE ARCHIVOS DE ENTRADA (PROBABILIDADES)
# -----------------------------------------------------------------------------
archivos_prob <- c(
  "/content/buckets/b1/exp/Comp3_Entrega-400/prediccion.txt",
  "/content/buckets/b1/exp/Comp3_Entrega-300/prediccion.txt"
)

# -----------------------------------------------------------------------------
# 2. CARGA Y ENSEMBLE (SOFT VOTING)
# -----------------------------------------------------------------------------
# Leemos solo lo necesario para no saturar la RAM
tbl_probs <- rbindlist(lapply(archivos_prob, function(x) {
  fread(x, select = c("numero_de_cliente", "prob"))
}))

# Calculamos el promedio por cliente
ensemble_final <- tbl_probs[, .(prob_promedio = mean(prob)), by = numero_de_cliente]

# -----------------------------------------------------------------------------
# 3. ORDENAMIENTO Y CORTE (RANKING)
# -----------------------------------------------------------------------------
# Ordenamos de mayor a menor probabilidad
setorder(ensemble_final, -prob_promedio)

# Creamos la columna Predicted llena de ceros (para todos los clientes)
ensemble_final[, Predicted := 0L]

# Asignamos 1 SOLAMENTE a los primeros 11250 (corte manual)
corte_seleccionado <- 11250
ensemble_final[1:corte_seleccionado, Predicted := 1L]

# -----------------------------------------------------------------------------
# 4. GENERACIÓN DEL ARCHIVO DE SALIDA
# -----------------------------------------------------------------------------
# Seleccionamos solo las columnas pedidas
archivo_salida <- ensemble_final[, .(numero_de_cliente, Predicted)]

# Guardamos el CSV
nombre_archivo <- "Comp3_Entrega_Ensemble_SoftVoting_Corte11250.csv"
fwrite(archivo_salida,
   file =  paste0("/content/buckets/b1/exp/Comp3_Entrega_Ensemble_SoftVoting_Corte11250.csv"),
    sep = ","
)

# -----------------------------------------------------------------------------
# 5. VALIDACIÓN FINAL (Visual)
# -----------------------------------------------------------------------------
cat("=== PROCESO FINALIZADO ===\n")
cat("Archivo generado:", nombre_archivo, "\n")
cat("Total de clientes en el archivo:", nrow(archivo_salida), "(Deben ser todos)\n")
cat("Total de envíos (Predicted = 1):", archivo_salida[Predicted == 1, .N], "\n")
cat("Corte utilizado:", corte_seleccionado, "\n")